my version of process data can be download here

https://drive.google.com/file/d/1Aw2Mg7BpVG5P7lw5_ABJsRku8AsAkJ7p/view?usp=sharing

this notebook if forked from 

https://www.kaggle.com/uomislab/instacart-xgboost-gridsearch-notebook

# import csv

In [2]:
orders = pd.read_csv('../input/instacart-market-basket-analysis/orders.csv' )
order_products_train = pd.read_csv('../input/instacart-market-basket-analysis/order_products__train.csv')
order_products_prior = pd.read_csv('../input/instacart-market-basket-analysis/order_products__prior.csv')
products = pd.read_csv('../input/instacart-market-basket-analysis/products.csv')

In [3]:
# check user and product numbers
len(set(orders['user_id'])), len(set(products['product_id']))

(206209, 49688)

In [4]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


# process data

## Create a DataFrame of product and user


In [5]:
#Merge the orders DF with order_products_prior by their order_id, keep only these rows with order_id that they are appear on both DFs
op_prior = orders.merge(order_products_prior, on='order_id', how='inner')
op_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [6]:
#Merge the orders DF with order_products_train by their order_id, keep only these rows with order_id that they are appear on both DFs
op_train = orders.merge(order_products_train, on='order_id', how='inner')
op_train.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [7]:
set(op_prior['eval_set']), set(op_train['eval_set'])

({'prior'}, {'train'})

In [8]:
# concat prior and training dataframe
op = pd.concat([op_prior, op_train])

op.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [9]:
op.tail()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
1384612,272231,206209,train,14,6,14,30.0,40603,4,0
1384613,272231,206209,train,14,6,14,30.0,15655,5,0
1384614,272231,206209,train,14,6,14,30.0,42606,6,0
1384615,272231,206209,train,14,6,14,30.0,37966,7,0
1384616,272231,206209,train,14,6,14,30.0,39216,8,1


## create features about user

In [10]:
user = op.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user.head()

,u_total_orders
user_id,
1,11
2,15
3,12
4,5
5,5


In [11]:
user = user.reset_index()
user.head()

,user_id,u_total_orders
0,1,11
1,2,15
2,3,12
3,4,5
4,5,5


In [12]:
# count how many product a user bought
user_count_bought_p = op.groupby('user_id')['order_id'].count().to_frame('u_total_bought_product')
user_count_bought_p.head()

,u_total_bought_product
user_id,
1,70
2,226
3,88
4,18
5,46


In [13]:
user = pd.merge(user, user_count_bought_p, on=['user_id'], how='left')

user.head()

,user_id,u_total_orders,u_total_bought_product
0,1,11,70
1,2,15,226
2,3,12,88
3,4,5,18
4,5,5,46


In [14]:
user['u_avg_bought_p_per_order'] = user.u_total_bought_product / user.u_total_orders

user.head()

,user_id,u_total_orders,u_total_bought_product,u_avg_bought_p_per_order
0,1,11,70,6.363636
1,2,15,226,15.066667
2,3,12,88,7.333333
3,4,5,18,3.600000
4,5,5,46,9.200000


## create features about user and product

In [15]:
uxp = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_total_bought')
uxp.head()

uxp_total_bought
user_id product_id                  
1       196                       11
        10258                     10
        10326                      1
        12427                     10
        13032                      4

In [16]:
uxp = uxp.reset_index()
uxp.head()

,user_id,product_id,uxp_total_bought
0,1,196,11
1,1,10258,10
2,1,10326,1
3,1,12427,10
4,1,13032,4


In [17]:
times = op.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
times.head()

Times_Bought_N
user_id product_id                
1       196                     11
        10258                   10
        10326                    1
        12427                   10
        13032                    4

In [18]:
total_orders = op.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders.head()

,total_orders
user_id,
1,11
2,15
3,12
4,5
5,5


In [19]:
first_order_no = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()

,user_id,product_id,first_order_number
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [20]:
span = pd.merge(total_orders, first_order_no, on='user_id', how='right')
span.head()

,user_id,total_orders,product_id,first_order_number
0,1,11,196,1
1,1,11,10258,2
2,1,11,10326,5
3,1,11,12427,1
4,1,11,13032,2


In [21]:
# The +1 includes in the difference the first order were the product has been purchased
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
span.head()

,user_id,total_orders,product_id,first_order_number,Order_Range_D
0,1,11,196,1,11
1,1,11,10258,2,10
2,1,11,10326,5,7
3,1,11,12427,1,11
4,1,11,13032,2,10


In [22]:
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D
0,1,196,11,11,1,11
1,1,10258,10,11,2,10
2,1,10326,1,11,5,7
3,1,12427,10,11,1,11
4,1,13032,4,11,2,10


In [23]:
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.Times_Bought_N / uxp_ratio.Order_Range_D
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D,uxp_reorder_ratio
0,1,196,11,11,1,11,1.000000
1,1,10258,10,11,2,10,1.000000
2,1,10326,1,11,5,7,0.142857
3,1,12427,10,11,1,11,0.909091
4,1,13032,4,11,2,10,0.400000



We keep only the 'user_id', 'product_id' and the final feature 'uxp_reorder_ratio'


In [24]:
uxp_ratio = uxp_ratio.drop(['Times_Bought_N', 'total_orders', 'first_order_number', 'Order_Range_D'], axis=1)
uxp_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.142857
3,1,12427,0.909091
4,1,13032,0.400000


In [25]:
del [times, first_order_no, span]

In [26]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')

del uxp_ratio
uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio
0,1,196,11,1.000000
1,1,10258,10,1.000000
2,1,10326,1,0.142857
3,1,12427,10,0.909091
4,1,13032,4,0.400000


In [27]:
uxp = uxp.merge(user, on=['user_id'], how='left')

uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,u_total_orders,u_total_bought_product,u_avg_bought_p_per_order
0,1,196,11,1.000000,11,70,6.363636
1,1,10258,10,1.000000,11,70,6.363636
2,1,10326,1,0.142857,11,70,6.363636
3,1,12427,10,0.909091,11,70,6.363636
4,1,13032,4,0.400000,11,70,6.363636


In [28]:
uxp['uxp_order_ratio'] = uxp.uxp_total_bought/uxp.u_total_orders 

uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,u_total_orders,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio
0,1,196,11,1.000000,11,70,6.363636,1.000000
1,1,10258,10,1.000000,11,70,6.363636,0.909091
2,1,10326,1,0.142857,11,70,6.363636,0.090909
3,1,12427,10,0.909091,11,70,6.363636,0.909091
4,1,13032,4,0.400000,11,70,6.363636,0.363636


In [29]:
uxp = uxp.drop(['uxp_total_bought', 'u_total_orders'], axis=1)
uxp.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio
0,1,196,1.000000,70,6.363636,1.000000
1,1,10258,1.000000,70,6.363636,0.909091
2,1,10326,0.142857,70,6.363636,0.090909
3,1,12427,0.909091,70,6.363636,0.909091
4,1,13032,0.400000,70,6.363636,0.363636


In [30]:
gc.collect()

269

In [31]:
uxp = uxp.fillna(0)
uxp.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio
0,1,196,1.000000,70,6.363636,1.000000
1,1,10258,1.000000,70,6.363636,0.909091
2,1,10326,0.142857,70,6.363636,0.090909
3,1,12427,0.909091,70,6.363636,0.909091
4,1,13032,0.400000,70,6.363636,0.363636


In [32]:
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797
53,6,test,1528013
74,7,train,525192
78,8,train,880375
82,9,train,1094988
88,10,train,1822501


In [33]:
# bring the info of the future orders to data DF
uxp = uxp.merge(orders_future, on='user_id', how='left')
uxp.head(10)

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,eval_set,order_id
0,1,196,1.000000,70,6.363636,1.000000,train,1187899
1,1,10258,1.000000,70,6.363636,0.909091,train,1187899
2,1,10326,0.142857,70,6.363636,0.090909,train,1187899
3,1,12427,0.909091,70,6.363636,0.909091,train,1187899
4,1,13032,0.400000,70,6.363636,0.363636,train,1187899
5,1,13176,0.200000,70,6.363636,0.181818,train,1187899
6,1,14084,0.090909,70,6.363636,0.090909,train,1187899
7,1,17122,0.142857,70,6.363636,0.090909,train,1187899
8,1,25133,1.000000,70,6.363636,0.818182,train,1187899
9,1,26088,0.272727,70,6.363636,0.272727,train,1187899


In [34]:
del orders_future

In [36]:
set(uxp['eval_set'])

{'test', 'train'}

In [38]:
len(set(uxp[uxp['eval_set'] == 'test']['user_id']))

75000

In [39]:
import numpy as np

#### check the range of my feature

In [40]:
np.max(uxp.uxp_reorder_ratio), np.min(uxp.uxp_reorder_ratio), np.mean(uxp.uxp_reorder_ratio), np.std(uxp.uxp_reorder_ratio)

(1.0, 0.01, 0.2911623067263651, 0.2773396550371502)

In [41]:
np.max(uxp.uxp_order_ratio), np.min(uxp.uxp_order_ratio), np.mean(uxp.uxp_order_ratio), np.std(uxp.uxp_order_ratio)

(1.0, 0.01, 0.14850959230850874, 0.15797293425175454)

# Make Y

In [42]:
# see if reordered before
uxp_reorder = op.groupby(['user_id', 'product_id'])['reordered'].max().to_frame('reordered')
uxp_reorder.head()

reordered
user_id product_id           
1       196                 1
        10258               1
        10326               0
        12427               1
        13032               1

In [43]:
uxp = uxp.merge(uxp_reorder, on=['user_id', 'product_id'], how='left')

# uxp = uxp.fillna(0)

uxp.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,eval_set,order_id,reordered
0,1,196,1.000000,70,6.363636,1.000000,train,1187899,1
1,1,10258,1.000000,70,6.363636,0.909091,train,1187899,1
2,1,10326,0.142857,70,6.363636,0.090909,train,1187899,0
3,1,12427,0.909091,70,6.363636,0.909091,train,1187899,1
4,1,13032,0.400000,70,6.363636,0.363636,train,1187899,1


In [45]:
uxp[uxp['eval_set'] == 'test'].head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,eval_set,order_id,reordered
140,3,248,0.090909,88,7.333333,0.083333,test,2774568,0
141,3,1005,0.333333,88,7.333333,0.083333,test,2774568,0
142,3,1819,0.333333,88,7.333333,0.250000,test,2774568,1
143,3,7503,0.100000,88,7.333333,0.083333,test,2774568,0
144,3,8021,0.090909,88,7.333333,0.083333,test,2774568,0


## 3. Merge into user item interaction table

In [46]:
uxp.to_csv('uxp.csv')